In [35]:
aoc_data = []
with open('example_18.txt') as f:
    for line in f:
        aoc_data.append(line.rstrip())

In [39]:
aoc_data = []
with open('data_18.txt') as f:
    for line in f:
        aoc_data.append(line.rstrip())

In [36]:
def flood_fill(field, r ,c, old, new):
    # we need the x and y of the start position, the old value,
    # and the new value
    # the flood fill has 4 parts
    # firstly, make sure the x and y are inbounds
    if not 0 <= r < len(field) or not 0 <= c < len(field[0]):
        return
    # secondly, check if the current position equals the old value
    if field[r][c] != old:
        return

    # thirdly, set the current position to the new value
    field[r][c] = new
    # fourthly, attempt to fill the neighboring positions
    flood_fill(field, r+1, c, old, new)
    flood_fill(field, r-1, c, old, new)
    flood_fill(field, r, c+1, old, new)
    flood_fill(field, r, c-1, old, new)

In [50]:
dirs = {
    'U': (-1,0),
    'D': (1,0),
    'L': (0,-1),
    'R': (0,1)
}

p2_dirs = {
    '3': (-1,0),
    '1': (1,0),
    '2': (0,-1),
    '0': (0,1)
}

turns = {
    ((1,0),(0,1)): 'LEFT',
    ((1,0),(0,-1)): 'RIGHT',
    ((-1,0),(0,1)): 'RIGHT',
    ((-1,0),(0,-1)): 'LEFT',
    ((0,1),(1,0)): 'RIGHT',
    ((0,1),(-1,0)): 'LEFT',
    ((0,-1),(1,0)): 'LEFT',
    ((0,-1),(-1,0)): 'RIGHT'
}

instrux = [line.split(' ') for line in aoc_data]
p2_instrux = []
for i, v in enumerate(instrux):
    instrux[i][0] = dirs[v[0]]
    instrux[i][1] = int(v[1])
    instrux[i][2] = v[2].replace('(', '').replace(')', '')

    p2_instrux.append([p2_dirs[v[2][-1:]], int(v[2][1:6], 16)])

trench_turns = {
    'RIGHT': 0,
    'LEFT': 0
}

grid_bounds = {
    'rc': [0,0],
    'max_r': 0,
    'min_r': 0,
    'max_c': 0,
    'min_c': 0
}

for i in range(len(instrux)):
    if i > 0:
        this_dir = instrux[i][0]
        prev_dir = instrux[i-1][0]
        turn_i = turns[(prev_dir, this_dir)]
        trench_turns[turn_i] += 1
    
    grid_bounds['rc'][0] += instrux[i][0][0] * instrux[i][1]
    grid_bounds['rc'][1] += instrux[i][0][1] * instrux[i][1]

    grid_bounds['max_r'] = max(grid_bounds['max_r'], grid_bounds['rc'][0])
    grid_bounds['min_r'] = min(grid_bounds['min_r'], grid_bounds['rc'][0])
    grid_bounds['max_c'] = max(grid_bounds['max_c'], grid_bounds['rc'][1])
    grid_bounds['min_c'] = min(grid_bounds['min_c'], grid_bounds['rc'][1])

# determine if perimeter turns left or right to create an enclosed shape
turn_direction = ''
turn_count = 0
for k, v in trench_turns.items():
    if v > turn_count:
        turn_direction = k
        turn_count = v

# based on whether path turns left or right, determine how to find inside of the wall based on wall direction
if turn_direction == 'RIGHT':
    inside_walls = {
        # direction: where inside wall is relative to rc
        (1,0): (0,-1),
        (-1,0): (0,1),
        (0,1): (1,0),
        (0,-1): (-1,0)
    }
else:
    inside_walls = {
        (1,0): (0,1),
        (-1,0): (0,-1),
        (0,1): (-1,0),
        (0,-1): (1,0)
    }

num_rows = 1 + grid_bounds['max_r'] + abs(grid_bounds['min_r'])
num_cols = 1 + grid_bounds['max_c'] + abs(grid_bounds['min_c'])


grid = []
for r in range(num_rows):
    grid.append([0] * num_cols)

start_rc = (grid_bounds['min_r'] * -1, grid_bounds['min_c'] * -1)

rc = list(start_rc)
grid[rc[0]][rc[1]] = 1
walls = {tuple(rc): (0,0)}
for instruct in instrux:
    for i in range(instruct[1]):
        rc[0] += instruct[0][0]
        rc[1] += instruct[0][1]
        grid[rc[0]][rc[1]] = 1
        walls[tuple(rc)] = instruct[0]

# for line in grid:
#     print(line)



In [51]:
import sys
sys.setrecursionlimit(1000)

for wall_rc, wall_dir in walls.items():
    try:
        inside_wall_dir = inside_walls[wall_dir]
        inside_wall = (wall_rc[0] + inside_wall_dir[0], wall_rc[1] + inside_wall_dir[1])
        if grid[inside_wall[0]][inside_wall[1]] == 0:
            flood_fill(grid, inside_wall[0], inside_wall[1], 0, 1)
    except:
        continue

p1_ans = 0
for row in grid:
    # print(row)
    for c in row:
        p1_ans += c
print(p1_ans)

40206
